<a href="https://colab.research.google.com/github/predictions01/zabbix/blob/main/RipartoDaSQLDBONL10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 from google.colab import drive
 drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [ ]:
main_folder = 'drive/MyDrive/Colab Notebooks/dati/'

In [ ]:
metrics = pd.read_pickle(main_folder+"zabbix-new2")
tickets = pd.read_pickle(main_folder+"tickets-new2")
device = 'SQLDBONL10'
target = 12

In [ ]:
import pickle
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
save_obj(tickets, 'tickets-AJ')

In [ ]:
tickets.to_csv('tickets-AJ2.csv', sep=';')

# Calcolo metriche

In [ ]:
my_metrics = metrics[metrics.device == device].set_index('timestamp')[['metric', 'value']]
df = my_metrics.pivot(columns='metric', values='value').dropna(thresh=my_metrics.index.nunique() * .9, axis=1).interpolate(limit_direction='both')
df['hour'] = df.index.hour
df['day'] = df.index.dayofweek

# Calcolo la y

In [ ]:
my_tickets = tickets[tickets.device == device].set_index('timestamp')
kappa = lambda x: int(any((x < ticket_time) and (ticket_time < x + pd.offsets.Hour(target)) for ticket_time in my_tickets.index))
df['y'] = df.index.map(kappa)

In [ ]:
from sklearn.model_selection import train_test_split

def balanced_train_test_split(df, test_size=.7):
    # Calcolo quant'è il 70% del numero di "1"
    num_tickets_training = int(df.y.sum() * test_size)

    # Faccio cumulative sum per considerare nel training set un numero di ticket = num_tickets_training
    cumsum = df.y.cumsum()

    # Conto quante righe ho che stanno dopo i primi <num_tickets_training> ticket
    num_val_tickets = cumsum[cumsum > num_tickets_training].shape[0]
    
    # Splitto con test_size fissato
    training_set, validation_set = train_test_split(df, test_size=num_val_tickets, shuffle=False)
    
    return training_set, validation_set

training_set, test_set = balanced_train_test_split(df)

In [ ]:
len(test_set[test_set.y == 1]) / len(test_set)

0.3034965034965035

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix, confusion_matrix

l = RandomForestClassifier().fit(training_set.loc[:, training_set.columns != 'y'], training_set.y)
l.score(test_set.loc[:, test_set.columns != 'y'], test_set.y)

0.7608391608391608

In [ ]:
confusion_matrix(test_set.y, l.predict(test_set.loc[:, test_set.columns != 'y']))
#plot_confusion_matrix(l, test_set.loc[:, test_set.columns != 'y'], test_set.y)

array([[443,  55],
       [116, 101]])

In [ ]:
df